In [9]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import wfdb.processing


In [ ]:
import os
import requests
import zipfile

def download_and_extract_mitdb(destination="data/mitdb"):
    url = "https://www.physionet.org/static/published-projects/mitdb/mit-bih-arrhythmia-database-1.0.0.zip"
    zip_path = "mitdb.zip"
    
    # Baixar o arquivo ZIP
    print("Baixando MIT-BIH Arrhythmia Database...")
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        with open(zip_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print("Download concluído.")
    else:
        print("Erro ao baixar o arquivo.")
        return

    # Extrair os arquivos
    print("Extraindo arquivos...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(destination)
    
    print(f"Arquivos extraídos para {destination}")
    
    # Remover o ZIP
    os.remove(zip_path)

# Executar a função
download_and_extract_mitdb()


Baixando MIT-BIH Arrhythmia Database...


In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import wfdb.processing

# Função para carregar o ECG de um paciente
def load_ecg(record_name="100"):
    record_path = f"data/mitdb/{record_name}"
    
    if not os.path.exists(f"{record_path}.dat"):
        print(f"Erro: Arquivo {record_name} não encontrado em {record_path}.")
        return None

    record = wfdb.rdrecord(record_path, sampto=3000)  # Lê até 3000 amostras
    signal = record.p_signal[:, 0]  # Pega apenas um canal

    return signal

# Filtro passa-baixa para remover ruídos de alta frequência
def lowpass_filter(signal_data, cutoff=20, fs=360, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return signal.filtfilt(b, a, signal_data)

# Detecção dos picos R usando a derivada da curva e limiares
def detect_r_peaks(signal_data, threshold=0.6):
    diff_signal = np.diff(signal_data)  # Derivada do sinal
    peaks, _ = signal.find_peaks(diff_signal, height=threshold)  # Detecta picos
    return peaks

signal = load_ecg()  # Carrega o sinal ECG
if signal is not None:
    filtered_signal = lowpass_filter(signal)  # Filtra o sinal
    r_peaks = detect_r_peaks(filtered_signal)  # Detecta os picos R

    # Plotando o ECG e os picos R detectados
    plt.figure(figsize=(12, 4))
    plt.plot(filtered_signal, label="Sinal Filtrado")
    plt.plot(r_peaks, filtered_signal[r_peaks], "ro", label="Picos R")
    plt.legend()
    plt.title("Detecção de Picos R no ECG")
    plt.xlabel("Amostras")
    plt.ylabel("Amplitude")
    plt.show()

NetFileNotFoundError: 404 Error: Not Found for url: https://physionet.org/files/mitdb/1.0.0/100